In [ ]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.1 MB/s eta 0:00:00


In [ ]:
pip install noisereduce

In [ ]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 11.7 MB/s eta 0:00:00


In [ ]:
pip install pydub

In [ ]:
pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=7261d9b9e0aa0847f1942f52e810b80adfca1ffde33f947115f832572c7df1e4
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
pip install webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73461 sha256=2a47587e300ca5d3af65b97a8b2ec60ec054983011d93a752de64d90faaa789a
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EncoderDecoderCache
from peft import get_peft_model, LoraConfig
import torch
import librosa
from datasets import Dataset
import re
import noisereduce as nr
from pydub import AudioSegment

In [ ]:
from google.colab import drive
# Mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the custom dataset
def load_custom_dataset_pip(data):
    audio_paths = []

    for sample in data:
        audio_path = sample["audio"]

        # Load audio using librosa
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})

    return Dataset.from_dict({"audio": audio_paths})

In [ ]:
def preprocess_function_pip(batch):
    # To hold the expanded input features and labels
    all_input_features = []

    # Process each example in the batch
    for i in range(len(batch["audio"])):
        # Access the audio array and corresponding text for the single example
        audio = batch["audio"][i]["array"]

        # Noise reduction, trimming, and normalization
        audio = nr.reduce_noise(y=audio, sr=16000)
        audio, _ = librosa.effects.trim(audio, top_db=20)
        audio = librosa.util.normalize(audio)

        # Set maximum length for a 30-second chunk
        sample_rate = 16000
        max_length = sample_rate * 30  # 30 seconds in samples
        overlap_duration = 1  # seconds
        overlap_samples = overlap_duration * sample_rate  # Convert overlap duration to samples

        # Create overlapping audio chunks
        audio_chunks = []
        for j in range(0, len(audio), max_length - 0):
            chunk = audio[j:j + max_length]
            if len(chunk) < max_length:  # If chunk is shorter than 30 seconds, pad it
                padding = max_length - len(chunk)
                chunk = np.pad(chunk, (0, padding), mode='constant')
            audio_chunks.append(chunk)

        # Process each audio-text chunk pair
        for audio_chunk in zip(audio_chunks):
            inputs = processor(audio_chunk, sampling_rate=sample_rate, padding=True,truncation=True, return_tensors="pt")  # Extract audio features

            # Append each chunk's features and labels to the expanded lists
            all_input_features.append(inputs.input_features[0].numpy())

    # Check if any features were collected
    print(f"Preprocessed {len(all_input_features)} Audio Chunk")
    # Return the expanded dictionary
    return {
        "input_features": all_input_features,
    }

In [ ]:
# @title
"""import torch
import evaluate
def generate_transcription_pip(batch):
    # Extract input features and create attention mask
    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension

    # Create an attention mask
    attention_mask = torch.ones(input_features.shape, dtype=torch.int)  # All input tokens are valid

    # Generate predicted IDs with the input features, attention mask, and forced language token
    predicted_ids = model.generate(
        input_features,
        attention_mask=attention_mask,
        language='en'  # Specify the target language for translation
    )

    # Decode the predicted token IDs back into text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription"""

'import torch\nimport evaluate\ndef generate_transcription_pip(batch):\n    # Extract input features and create attention mask\n    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension\n\n    # Create an attention mask\n    attention_mask = torch.ones(input_features.shape, dtype=torch.int)  # All input tokens are valid\n\n    # Generate predicted IDs with the input features, attention mask, and forced language token\n    predicted_ids = model.generate(\n        input_features,\n        attention_mask=attention_mask,\n        language=\'en\'  # Specify the target language for translation\n    )\n\n    # Decode the predicted token IDs back into text\n    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]\n\n    return transcription'

In [ ]:
# @title
"""import torch
import evaluate

def generate_transcription_pip(batch):
    # Extract input features and create attention mask
    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension

    # Create an attention mask, assuming no padding in input features
    attention_mask = torch.ones(input_features.shape, dtype=torch.int)  # All input tokens are valid

    # Generate predicted IDs with the input features and attention mask
    predicted_ids = model.generate(
        input_features,
        attention_mask=attention_mask,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="en")  # Use processor to set language tokens
    )

    # Decode the predicted token IDs back into text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription"""

'import torch\nimport evaluate\n\ndef generate_transcription_pip(batch):\n    # Extract input features and create attention mask\n    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension\n\n    # Create an attention mask, assuming no padding in input features\n    attention_mask = torch.ones(input_features.shape, dtype=torch.int)  # All input tokens are valid\n\n    # Generate predicted IDs with the input features and attention mask\n    predicted_ids = model.generate(\n        input_features,\n        attention_mask=attention_mask,\n        forced_decoder_ids=processor.get_decoder_prompt_ids(language="en")  # Use processor to set language tokens\n    )\n\n    # Decode the predicted token IDs back into text\n    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]\n\n    return transcription'

In [ ]:
import torch

def generate_transcription_pip(batch):
    # Extract input features from the batch and create attention mask
    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension

    # Create attention mask to ensure model distinguishes padding from meaningful input
    attention_mask = torch.ones(input_features.shape)  # Initialize attention mask with 1s
    attention_mask[input_features == processor.feature_extractor.padding_value] = 0  # Set to 0 where there's padding

    # Generate predicted token IDs with the input features and attention mask
    predicted_ids = model.generate(
        input_features,
        attention_mask=attention_mask,  # Pass the attention mask
        language="en",  # Specify language directly here to avoid conflict
        max_new_tokens=444,  # Set a reasonable limit on tokens to generate
        no_repeat_ngram_size=3,  # To avoid repeated phrases in transcription
        num_beams=5,  # For better decoding (optional)
    )

    # Decode the predicted token IDs back into text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription

# T3 Model Pipeline

In [ ]:
def T3_pipeline(input_path,model,processor):

  #audio = AudioSegment.from_mp3(input_path)
  #wav_filename = file.filename.replace('.mp3', '.wav')
  #wav_file_path = os.path.join("temp", wav_filename)
  #audio.export(wav_file_path, format='wav')
  audio = AudioSegment.from_mp3(mp3_path)
  wav_path = '/content/Andrew Ng_test.wav'
  audio.export(wav_path, format="wav")
  print("Conversion complete. Saved as:", wav_path)
  data_in = [
    {"audio": wav_path},
  ]
  input = load_custom_dataset_pip(data_in)
  input_prep = input.map(preprocess_function_pip, batched=True, remove_columns=input.column_names)
  results = input_prep.map(lambda batch: {"prediction": generate_transcription_pip(batch)})
  text = results.to_pandas()
  predictions = text['prediction'].tolist()
  result_str = "\n".join(predictions)

  return result_str

In [ ]:
mp3_path = '/content/احلى test عليك.wav'

# Path to the saved fine-tuned model on Google Drive
model_path = "/content/drive/MyDrive/whisper_finetuned_V2"

# Load the fine-tuned Whisper model from the saved directory
model = WhisperForConditionalGeneration.from_pretrained(model_path)
# Load the processor (tokenizer + feature extractor)
processor = WhisperProcessor.from_pretrained(model_path)

Transcription,results = T3_pipeline(mp3_path,model,processor)

Conversion complete. Saved as: /content/Andrew Ng_test.wav


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Preprocessed 3 Audio Chunk


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
print(Transcription)

 this entire animated video is made with ai. stick around and i'll show you how to create your own without any tech skills. i just can't believe how unfair life is sometimes. i work hard, but nothing goes my way. remember the story of the bamboo and the fern? the furn grew quickly, but the bamboo took years to sprout. so what are you saying? when the bamboo did sprout, it grew strong and tall. our time will come, alex. we just need patience. but it's so hard to wait. i want things to get better.
 i know it's hard, but good things take time. just like the bamboo, you will grow strong too. i guess you're right. maybe it's just not my season yet. exactly. keep going. your bamboo is just about to sprout. thanks, jamie. i feel better now. you see, creating videos like these can cost thousands of dollars. require a fancy animation degree, and involve complicated software. what if i told you anyone can create a pixar style animated video by simply typing a text? that's right. no fancy animati